## 1. Configuração e Imports

In [ ]:
import json
import sys
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Configuração de caminhos
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

from src.core.config import get_paths

paths = get_paths()
PROCESSED_DIR = paths.data_processed
OUTPUTS_DIR = paths.root / "data" / "outputs"

print(f"Project Root: {PROJECT_ROOT}")

## 2. Carregamento de Dados

In [ ]:
# Carregar dados
with open(PROCESSED_DIR / "fundamentals.json", "r") as f:
    fundamentals = json.load(f)

with open(PROCESSED_DIR / "capm_results.json", "r") as f:
    capm = json.load(f)

with open(PROCESSED_DIR / "qval_results.json", "r") as f:
    qval = json.load(f)

print("Dados carregados:")
print(f"  - Ticker: {fundamentals['metadata']['ticker']}")
print(f"  - Preço: R$ {fundamentals['market_data']['price']:.2f}")
print(f"  - Ke (CAPM): {capm['ke_capm']*100:.2f}%")
print(f"  - Q-VAL Score: {qval['score_final']:.1f}/100")

## 3. Benchmarks de Referência

In [ ]:
# Múltiplos históricos PETR4 (média 5 anos - 2020-2024)
HISTORICAL_MULTIPLES = {
    "price_earnings": {"mean": 6.5, "std": 3.2},
    "ev_ebitda": {"mean": 3.8, "std": 1.2},
    "price_to_book": {"mean": 1.1, "std": 0.3},
    "dividend_yield": {"mean": 12.0, "std": 8.0},
}

# Múltiplos setoriais (Oil & Gas Integrated - Brasil)
SECTOR_MULTIPLES = {
    "price_earnings": {"mean": 8.0, "std": 4.0},
    "ev_ebitda": {"mean": 4.5, "std": 1.5},
    "price_to_book": {"mean": 1.2, "std": 0.4},
    "dividend_yield": {"mean": 8.0, "std": 5.0},
}

print("Benchmarks definidos.")

## 4. Análise de Múltiplos Relativos

Comparamos os múltiplos atuais da PETR4 contra:
- Média histórica dos últimos 5 anos
- Média do setor Oil & Gas

In [ ]:
# Múltiplos atuais
valuation = fundamentals["valuation_metrics"]
dividend = fundamentals["dividend_data"]

current_multiples = {
    "price_earnings": valuation["price_earnings"],
    "ev_ebitda": valuation["enterprise_to_ebitda"],
    "price_to_book": valuation["price_to_book"],
    "dividend_yield": dividend["dividend_yield"],
}

# Análise comparativa
analysis = []
for metric, value in current_multiples.items():
    hist = HISTORICAL_MULTIPLES[metric]
    sect = SECTOR_MULTIPLES[metric]
    
    premium_hist = (value / hist["mean"] - 1) * 100
    premium_sect = (value / sect["mean"] - 1) * 100
    
    analysis.append({
        "Métrica": metric.replace("_", " ").title(),
        "Atual": value,
        "Histórico": hist["mean"],
        "Setor": sect["mean"],
        "vs. Hist.": f"{premium_hist:+.1f}%",
        "vs. Setor": f"{premium_sect:+.1f}%",
    })

df_multiples = pd.DataFrame(analysis)
display(df_multiples)

In [ ]:
# Visualização de múltiplos
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

metrics = ["price_earnings", "ev_ebitda", "price_to_book", "dividend_yield"]
titles = ["P/L", "EV/EBITDA", "P/VP", "Dividend Yield (%)"]

for ax, metric, title in zip(axes.flat, metrics, titles):
    current = current_multiples[metric]
    hist = HISTORICAL_MULTIPLES[metric]["mean"]
    sect = SECTOR_MULTIPLES[metric]["mean"]
    
    bars = ax.bar(["Atual", "Histórico", "Setor"], [current, hist, sect],
                  color=["#3498db", "#95a5a6", "#7f8c8d"], edgecolor="black")
    
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.2f}', xy=(bar.get_x() + bar.get_width()/2, height),
                   xytext=(0, 3), textcoords="offset points", ha='center', fontsize=10)
    
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.set_ylabel("Valor")
    ax.yaxis.grid(True, linestyle='--', alpha=0.5)

plt.suptitle("Análise de Múltiplos — PETR4", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 5. Modelo de Gordon (Dividend Discount Model)

O Modelo de Gordon calcula o valor justo de uma ação com base em dividendos:

$$P_0 = \frac{D_1}{K_e - g}$$

Onde:
- $D_1$ = Dividendo esperado no próximo período
- $K_e$ = Custo de capital próprio (CAPM)
- $g$ = Taxa de crescimento sustentável dos dividendos

In [ ]:
# Parâmetros
market = fundamentals["market_data"]
profitability = fundamentals["profitability_metrics"]
growth = fundamentals["growth_metrics"]

current_price = market["price"]
dy = dividend["dividend_yield"] / 100  # Converter para decimal
roe = profitability["return_on_equity"]
ke = capm["ke_capm"]

# Estimativa de Payout Ratio
pe = valuation["price_earnings"]
payout_ratio = min(0.95, dy * pe)

# Taxa de crescimento (g)
# Método 1: g = ROE × (1 - Payout)
g_roe = roe * (1 - payout_ratio)

# Método 2: Crescimento histórico de earnings
g_earnings = max(-0.10, min(0.15, growth["earnings_growth"]))

# Média (com piso em 0%)
g = max(0.0, (g_roe + g_earnings) / 2)

print(f"Parâmetros do Modelo de Gordon:")
print(f"  Dividend Yield atual: {dy*100:.2f}%")
print(f"  ROE: {roe*100:.2f}%")
print(f"  Payout Ratio (est.): {payout_ratio*100:.2f}%")
print(f"  g (via ROE): {g_roe*100:.2f}%")
print(f"  g (via earnings): {g_earnings*100:.2f}%")
print(f"  g (média): {g*100:.2f}%")
print(f"  Ke (CAPM): {ke*100:.2f}%")

In [ ]:
# D1 = D0 × (1 + g)
d0_per_share = current_price * dy
d1 = d0_per_share * (1 + g)

# Valor Justo
if ke > g:
    fair_value = d1 / (ke - g)
else:
    fair_value = d1 / 0.10  # Fallback

# Upside/Downside
upside = (fair_value / current_price - 1) * 100

print(f"\nResultados do Modelo de Gordon:")
print(f"  D0 (dividendo atual/ação): R$ {d0_per_share:.2f}")
print(f"  D1 (dividendo esperado):   R$ {d1:.2f}")
print(f"  Valor Justo (P0):          R$ {fair_value:.2f}")
print(f"  Preço Atual:               R$ {current_price:.2f}")
print(f"  Upside/Downside:           {upside:+.1f}%")

## 6. Implied Cost of Capital (ICC)

O ICC é calculado por engenharia reversa do Gordon:

$$ICC = \frac{D_1}{P_0} + g$$

Se ICC > Ke → Ativo está **subprecificado** (mercado exige retorno maior que o justo)  
Se ICC < Ke → Ativo está **sobreprecificado** (mercado exige retorno menor que o justo)

In [ ]:
# Método 1: ICC via Gordon reverso
icc_gordon = (d1 / current_price) + g

# Método 2: ICC via Earnings Yield + g
ey = valuation["earnings_yield"]
icc_ey = ey + g

# Média ponderada
icc = 0.6 * icc_gordon + 0.4 * icc_ey

# Spread
spread = icc - ke

print(f"Implied Cost of Capital (ICC):")
print(f"  ICC (Gordon):         {icc_gordon*100:.2f}%")
print(f"  ICC (Earnings Yield): {icc_ey*100:.2f}%")
print(f"  ICC (Ponderado):      {icc*100:.2f}%")
print(f"  Ke (CAPM):            {ke*100:.2f}%")
print(f"  Spread (ICC - Ke):    {spread*100:+.2f}%")

## 7. Diagnóstico de Mispricing

In [ ]:
# Classificação
if spread > 0.02:
    classification = "Fortemente Subprecificado"
    signal = "Compra Forte"
    emoji = "🟢🟢"
elif spread > 0.005:
    classification = "Subprecificado"
    signal = "Compra"
    emoji = "🟢"
elif spread > -0.005:
    classification = "Precificação Justa"
    signal = "Neutro"
    emoji = "🟡"
elif spread > -0.02:
    classification = "Sobreprecificado"
    signal = "Venda"
    emoji = "🔴"
else:
    classification = "Fortemente Sobreprecificado"
    signal = "Venda Forte"
    emoji = "🔴🔴"

print(f"\n{'='*50}")
print(f"DIAGNÓSTICO DE MISPRICING")
print(f"{'='*50}")
print(f"\n📊 Classificação: {classification}")
print(f"📌 Sinal ICC:     {emoji} {signal}")
print(f"📈 Spread:        {spread*100:+.2f}%")

## 8. Integração com Q-VAL

In [ ]:
# Sinais de cada metodologia
qval_score = qval["score_final"]
qval_signal = qval["recommendation"]

print(f"\n{'='*50}")
print(f"CONSOLIDAÇÃO DE SINAIS")
print(f"{'='*50}")
print(f"\n1️⃣ ANÁLISE DE MÚLTIPLOS:")
print(f"   P/L: {current_multiples['price_earnings']:.2f}x vs. histórico {HISTORICAL_MULTIPLES['price_earnings']['mean']:.2f}x")
print(f"   → {'📉 DESCONTO' if current_multiples['price_earnings'] < HISTORICAL_MULTIPLES['price_earnings']['mean'] else '📈 PRÊMIO'}")

print(f"\n2️⃣ MODELO DE GORDON:")
print(f"   Valor Justo: R$ {fair_value:.2f}")
print(f"   Preço Atual: R$ {current_price:.2f}")
print(f"   → {'🟢 SUBAVALIADO' if upside > 5 else ('🔴 SOBREAVALIADO' if upside < -5 else '🟡 JUSTO')} ({upside:+.1f}%)")

print(f"\n3️⃣ IMPLIED COST OF CAPITAL:")
print(f"   ICC: {icc*100:.2f}%")
print(f"   Ke:  {ke*100:.2f}%")
print(f"   → {emoji} {signal}")

print(f"\n4️⃣ Q-VAL SCORE:")
print(f"   Score: {qval_score:.1f}/100")
print(f"   → {qval_signal}")

In [ ]:
# Recomendação Final Consolidada
signal_map = {
    "Compra Forte": 1.0,
    "Compra": 0.5,
    "Neutro": 0.0,
    "Venda": -0.5,
    "Venda Forte": -1.0,
}

icc_signal_val = signal_map.get(signal, 0)
qval_signal_val = signal_map.get(qval_signal, 0)
gordon_signal_val = 0.5 if upside > 10 else (-0.5 if upside < -10 else 0)

consolidated = (icc_signal_val + qval_signal_val + gordon_signal_val) / 3

if consolidated > 0.4:
    final_rec = "COMPRA"
    final_emoji = "🟢"
elif consolidated > -0.4:
    final_rec = "NEUTRO"
    final_emoji = "🟡"
else:
    final_rec = "VENDA"
    final_emoji = "🔴"

print(f"\n{'='*50}")
print(f"🏆 RECOMENDAÇÃO FINAL CONSOLIDADA")
print(f"{'='*50}")
print(f"\n   Score consolidado: {consolidated:+.2f}")
print(f"   \n   >>> {final_emoji} {final_rec} <<<")
print(f"\n{'='*50}")

## 9. Visualização: ICC vs CAPM

In [ ]:
# Gráfico ICC vs Ke
fig, ax = plt.subplots(figsize=(10, 6))

categories = ['Ke (CAPM)', 'ICC\n(Gordon)', 'ICC\n(Earn. Yield)', 'ICC\n(Ponderado)']
values = [ke * 100, icc_gordon * 100, icc_ey * 100, icc * 100]
colors = ['#2ecc71', '#3498db', '#9b59b6', '#e74c3c']

bars = ax.bar(categories, values, color=colors, edgecolor='black', linewidth=1.2)

for bar, val in zip(bars, values):
    height = bar.get_height()
    ax.annotate(f'{val:.2f}%',
               xy=(bar.get_x() + bar.get_width() / 2, height),
               xytext=(0, 5),
               textcoords="offset points",
               ha='center', va='bottom',
               fontsize=12, fontweight='bold')

ax.axhline(y=ke*100, color='#2ecc71', linestyle='--', linewidth=2, alpha=0.7, label=f'Ke CAPM = {ke*100:.2f}%')
ax.set_ylabel('Taxa de Retorno (%)', fontsize=12)
ax.set_title('Custo de Capital Implícito (ICC) vs. CAPM — PETR4', fontsize=14, fontweight='bold')
ax.set_ylim(0, max(values) * 1.2)
ax.yaxis.grid(True, linestyle='--', alpha=0.5)
ax.legend(loc='upper right')

plt.tight_layout()
plt.show()

## 10. Sumário Executivo

In [ ]:
print("\n" + "=" * 70)
print("SUMÁRIO EXECUTIVO - ANÁLISE DE VALUATION PETR4")
print("=" * 70)

print(f"\n📅 Data: {datetime.now().strftime('%d/%m/%Y')}")
print(f"📈 Ticker: PETR4")
print(f"💰 Preço Atual: R$ {current_price:.2f}")

print("\n" + "-" * 70)
print("MÚLTIPLOS")
print("-" * 70)
print(f"  P/L:           {current_multiples['price_earnings']:.2f}x (histórico: {HISTORICAL_MULTIPLES['price_earnings']['mean']:.2f}x)")
print(f"  EV/EBITDA:     {current_multiples['ev_ebitda']:.2f}x (histórico: {HISTORICAL_MULTIPLES['ev_ebitda']['mean']:.2f}x)")
print(f"  Dividend Yield: {current_multiples['dividend_yield']:.2f}% (histórico: {HISTORICAL_MULTIPLES['dividend_yield']['mean']:.2f}%)")

print("\n" + "-" * 70)
print("MODELO DE GORDON")
print("-" * 70)
print(f"  Taxa de Crescimento (g): {g*100:.2f}%")
print(f"  Custo de Capital (Ke):   {ke*100:.2f}%")
print(f"  Valor Justo:             R$ {fair_value:.2f}")
print(f"  Upside/Downside:         {upside:+.1f}%")

print("\n" + "-" * 70)
print("MISPRICING")
print("-" * 70)
print(f"  ICC (Ponderado): {icc*100:.2f}%")
print(f"  Ke (CAPM):       {ke*100:.2f}%")
print(f"  Spread:          {spread*100:+.2f}%")
print(f"  Diagnóstico:     {classification}")

print("\n" + "-" * 70)
print("RECOMENDAÇÃO CONSOLIDADA")
print("-" * 70)
print(f"  Q-VAL Score:       {qval_score:.1f}/100 ({qval_signal})")
print(f"  Sinal ICC:         {signal}")
print(f"  Gordon DDM:        {upside:+.1f}%")
print(f"\n  >>> {final_emoji} RECOMENDAÇÃO FINAL: {final_rec} <<<")

print("\n" + "=" * 70)

## 11. Arquivos Gerados

In [ ]:
# Verificar arquivos gerados
print("Arquivos gerados pela Fase 6:")
print(f"  ✅ {PROCESSED_DIR / 'valuation_results.json'}")
print(f"  ✅ {OUTPUTS_DIR / 'tables' / 'valuation_multiplos.tex'}")
print(f"  ✅ {OUTPUTS_DIR / 'tables' / 'diagnostico_mispricing.tex'}")
print(f"  ✅ {OUTPUTS_DIR / 'figures' / 'icc_vs_capm.pdf'}")
print(f"  ✅ {OUTPUTS_DIR / 'figures' / 'valuation_summary.pdf'}")